# 📍 GridWorld - Value Iteration Algorithm

This notebook implements **Value Iteration** to find the optimal policy for a **4x4 GridWorld environment**.  
The agent starts at (0,0) and must reach the goal (🏁) at (3,3), avoiding obstacles (█) and penalty zones (🔥).

## 🏆 Rules:
- The agent can move **Up (↑), Down (↓), Left (←), or Right (→)**.
- Each step has a **default reward of -1** (to encourage a shorter path).
- The **goal square gives +10** and ends the episode.
- **Obstacles (█) are impassable**.
- **Penalty squares (🔥) give -3** if stepped on.
- The agent uses **Value Iteration** to compute the best path.

# 🔢 Understanding Value Iteration

Value Iteration is a **Dynamic Programming algorithm** that updates the value of each state using:

V(s) = max_a( R(s,a) + gamma * V(s'))

Where:
- \( V(s) \) = Value of state \( s \)
- \( R(s, a) \) = Reward for taking action \( a \) from state \( s \)
- \( \gamma \) = Discount factor (importance of future rewards)
- \( s' \) = New state after taking action \( a \)

### 📌 Steps:
1. **Initialize** all state values to zero.
2. **Iterate until convergence**:
   - Update each state's value based on the best possible future reward.
3. **Extract the best policy**:
   - For each state, pick the action that leads to the highest value.


In [44]:
import numpy as np

# Paramètres de l'environnement
GRID_SIZE = 4
GAMMA = 0.9  # Facteur de réduction
THETA = 1e-4  # Seuil de convergence

# Récompenses et obstacles
REWARDS = np.full((GRID_SIZE, GRID_SIZE), -1)  # Pénalité de déplacement
REWARDS[3, 3] = 10  # Récompense pour la case objectif
REWARDS[2, 2] = -3  # Case de pénalité ajoutée
OBSTACLES = [(1, 1)]  # Cases infranchissables

# Position initiale de l'agent
START_POS = (0, 0)

# Directions de mouvement
ACTIONS = {
    '↑': (-1, 0),
    '↓': (1, 0),
    '←': (0, -1),
    '→': (0, 1)
}

# Initialisation de la valeur des états
V = np.zeros((GRID_SIZE, GRID_SIZE))


def is_valid(state):
    """Vérifie si un état est valide (dans la grille et pas un obstacle)."""
    x, y = state
    return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and (x, y) not in OBSTACLES

def value_iteration():
    """Itère sur les valeurs des états pour trouver les meilleures actions."""
    global V
    while True:
        delta = 0
        new_V = np.copy(V)
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if (x, y) == (3, 3) or (x, y) in OBSTACLES:
                    continue  # Ne pas mettre à jour l'objectif ni les obstacles
                
                values = []
                for action, (dx, dy) in ACTIONS.items():
                    new_x, new_y = x + dx, y + dy
                    if is_valid((new_x, new_y)):
                        values.append(REWARDS[x, y] + GAMMA * V[new_x, new_y])  # Utiliser la case actuelle pour la récompense
                    else:
                        values.append(REWARDS[x, y] + GAMMA * V[x, y])  # Reste sur place si invalide
                
                new_V[x, y] = max(values)  # Choisir la meilleure action
                delta = max(delta, abs(new_V[x, y] - V[x, y]))
        
        V = new_V
        if delta < THETA:
            break

def get_policy():
    """Retourne une politique optimisée sous forme de flèches."""
    policy = np.full((GRID_SIZE, GRID_SIZE), ' ', dtype=str)
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if (x, y) == (3, 3):
                policy[x, y] = '🏁'  # Arrivée
                continue
            if (x, y) in OBSTACLES:
                policy[x, y] = '█'  # Mur
                continue
            if (x, y) == (2, 2):
                policy[x, y] = '🔥'  # Case de pénalité
                continue
            
            best_action = None
            best_value = float('-inf')
            for action, (dx, dy) in ACTIONS.items():
                new_x, new_y = x + dx, y + dy
                if is_valid((new_x, new_y)) and V[new_x, new_y] > best_value:
                    best_value = V[new_x, new_y]
                    best_action = action
            
            policy[x, y] = best_action if best_action else ' '
    
    # Ajout de l'agent 🚀 à la case de départ
    policy[START_POS] = '🚀'
    
    return policy

# Exécuter l'algorithme et afficher la politique optimale
value_iteration()
policy = get_policy()


# 📊 Results - Optimal Policy

- The agent (🚀) starts at (0,0).
- The goal (🏁) is at (3,3).
- Walls (█) block movement.
- A penalty zone (🔥) at (2,2) should be avoided.

## ✅ Expected Output:


In [39]:
print("\n".join([" ".join(row) for row in policy]))

🚀 → ↓ ↓
↓ █ → ↓
↓ ↓ 🔥 ↓
→ → → 🏁
